# Chat Completions API

Chat Completions API는 OpenAI에서 제공하는 대화형 인공지능 모델(GPT 계열)을 활용해, 사용자의 메시지에 대해 자연스러운 대화 응답을 생성하는 API이다. 이 API는 챗봇, AI 비서, 자동화된 상담 시스템 등 다양한 대화형 서비스에 적용할 수 있다.


- **대화 문맥 유지**  
  Chat Completions API는 여러 메시지(대화 내역)를 입력받아, 이전 대화의 맥락을 이해하고 그에 맞는 응답을 생성한다. 즉, 단순히 한 문장만을 이어 쓰는 것이 아니라, 대화의 흐름을 반영하여 자연스러운 대화를 이어갈 수 있다.

- **역할(Role) 기반 메시지 구조**  
  입력 메시지는 배열 형태로 전달하며, 각 메시지는 `role`과 `content`로 구성된다.  
  - `system`: AI의 태도, 성격, 역할을 정의(예: "너는 친절한 도우미야.")
  - `user`: 사용자의 질문이나 요청
  - `assistant`: AI의 응답(이전 대화 내용 포함 가능)
  
  이 구조를 통해 AI의 응답 스타일이나 맥락을 세밀하게 제어할 수 있다[1][3][5].

**주요 파라미터 설명**

| 파라미터        | 설명                                                                 |
|----------------|----------------------------------------------------------------------|
| model          | 사용할 언어 모델명 (예: gpt-3.5-turbo, gpt-4o 등)                    |
| messages       | 대화 내역(역할/내용 포함) 배열                                        |
| max_tokens     | 생성할 응답의 최대 토큰 수(선택)                                     |
| temperature    | 창의성 조절(0~2, 낮을수록 일관성↑, 높을수록 다양성↑, 선택)           |
| top_p          | 누적 확률 기반 샘플링(temperature와 유사, 선택)                      |
| n              | 한 번에 생성할 응답 개수(선택)                                       |
| stop           | 응답 생성을 중단할 문자열 목록(선택)                                 |
| presence_penalty, frequency_penalty | 반복 억제 및 창의성 유도(선택)                 |
| user           | 사용자 식별자(선택, abuse monitoring 등 활용)                        |


- 위 예시에서 `messages` 배열에는 대화의 모든 메시지가 순서대로 들어가야 한다.  
- OpenAI는 이전 요청을 기억하지 않기 때문에, 매 API 호출마다 대화 내역 전체를 함께 보내야 한다.

In [2]:
from google.colab import userdata
import os
from openai import OpenAI

OPENAI_API_KEY = userdata.get('20250624_skn14_llm')
client = OpenAI(api_key = OPENAI_API_KEY)

#환경변수 지정
# os.environ['OPENAI_API_KEY'] = userdata.get('20250624_skn14_llm')
# client = OpenAI()

### 대화형 챗봇

In [10]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {'role':'system',
        'content': '너는 친절한 챗봇입니다.'},
        {'role': 'user',
         'content': '안녕, 내 이름은 차은우야'},
        {'role': 'assistant',
         'content': '안녕하세요, 차은우님! 어떻게 도와드릴까요?'},
         {'role': 'user',
         'content': '잘 지냈어? 내 이름 기억해?'},
    ]
)

print(response.choices[0].message.content)

네, 차은우님! 잘 지냈습니다. 차은우님께서 어떤 이야기나 질문이 있으신가요?


In [15]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [
        {'role':'system',
        'content': '너는 LLM 전문가입니다.'},
        {'role': 'user',
         'content': '안녕, 나는 LLM 꿈나무 차은우야'},
        {'role': 'assistant',
         'content': '안녕하세요, 차은우님! LLM 꿈나무라니 멋진 목표를 가지고 계시네요. LLM에 대해 어떤 궁금증이나 배우고 싶은 부분이 있나요? 도와드릴 수 있는 게 있다면 말씀해 주세요!'},
        {'role': 'user',
         'content': 'Transformer 모델을 공부하고 싶어'},
        {'role': 'assistant',
         'content': """
         좋아요! Transformer 모델은 자연어 처리(NLP)에서 매우 중요한 구조로, 특히 BERT, GPT와 같은 최신 모델들의 기반이 되고 있습니다. Transformer의 기본 개념과 구성 요소에 대해 간략히 설명해 드릴게요.

### 1. Transformer의 기본 구조
Transformer 모델은 크게 인코더와 디코더로 나뉩니다. 대부분의 현대 NLP 모델(예: BERT)은 인코더만 사용하고, GPT는 디코더만 사용합니다.

- **인코더**: 입력 문장을 처리하여 입력의 의미를 포착하는 역할을 합니다. 여러 개의 인코더 블록으로 구성되어 있습니다.
- **디코더**: 인코더에서의 출력을 바탕으로 새로운 문장을 생성합니다. (예: 번역기)

### 2. 주요 구성 요소
- **Self-Attention**: 입력의 각 단어가 다른 단어에 얼마나 집중해야 하는지를 계산합니다. 이 방식은 단어의 의미를 더 잘 이해할 수 있도록 도와줍니다.

- **Multi-Head Attention**: 여러 개의 Self-Attention 메커니즘을 동시에 사용하여 정보를 다양한 관점에서 캡처합니다.

- **Feed Forward Neural Network**: 각 인코더 또는 디코더 블록 내에서 사용되는 완전 연결 네트워크입니다. 일반적으로 두 개의 선형 변환과 활성화 함수(예: ReLU)를 사용합니다.

- **Position-wise Feed-Forward Networks**: 입력의 각 위치에 대해 독립적으로 작용합니다.

- **Positional Encoding**: Transformer는 순서를 직접적으로 반영하지 않기 때문에, 단어의 위치 정보를 추가하기 위해 포지셔널 인코딩이 사용됩니다.

- **Layer Normalization**: 각 블록의 출력을 정규화하여 학습을 안정화합니다.

### 3. 학습 방법
Transformer 모델은 대규모 데이터셋에서 학습하여 패턴을 찾고, 예측력을 높입니다. 일반적으로 위탁 학습 방식을 사용하며, 무수히 많은 문서에서 상황에 맞는 응답이나 단어를 조합하는 법을 배웁니다.

### 4. 참고 자료
- "Attention is All You Need" 논문: Transformer의 원조 논문으로, 기본 개념이 설명되어 있습니다.
- 다양한 오픈소스 라이브러리: Hugging Face의 Transformers 라이브러리에서는 이미 제공되는 Transformer 모델을 쉽게 사용하고 실험할 수 있습니다.

더 궁금한 부분이나 구체적인 질문이 있다면 자유롭게 물어보세요!

         """},
    {'role': 'user',
     'content': '초등학생도 이해할 수 있도록 쉽게 설명해줘'}]
)

print(response.choices[0].message.content)

물론이에요! 초등학생도 이해할 수 있도록 쉽게 설명해볼게요.

### Transformer 모델이란?

Transformer는 컴퓨터가 글을 읽고 이해하고, 답을 만들어내는 방법 중 하나예요. 마치 로봇이 우리랑 대화하는 것처럼요. 자, 그럼 어떻게 작동하는지 한 번 살펴볼까요?

### 1. 여러 사람의 대화처럼 생각해보세요

상상해보세요. 여러분이 친구들과 이야기를 나누고 있는데, 이야기의 주제와 관련 있는 다른 친구들이 여러분의 이야기에 귀를 기울이고 있다고 해요. 여기서 중요한 것은 **각 친구가 서로의 이야기에 주의를 기울이는 것**이에요. 이걸 **"Attention"**이라고 해요.

### 2. 단어들을 잘 이해하기 위해

이 모델은 글을 읽을 때 각 단어가 다른 단어와 어떻게 연결되는지를 스스로 알아낼 수 있어요. 예를 들어 “고양이가 쥐를 쫓아갔다”라는 문장에서 “고양이”와 “쥐”의 관계를 이해하는 거죠. 이때 여러 친구가 각자 다른 이야기를 듣고 어떤 정보를 얻는 것처럼 **여러 개의 ‘눈’(multi-head attention)**으로 보게 돼요.

### 3. 순서도 기억해요

하지만 단어들이 나오는 순서도 중요하잖아요? “나는 사과를 먹었다”와 “사과를 나는 먹었다”는 조금 다른 의미를 가져요. 그래서 Transformer는 각 단어에 ‘위치 정보’를 추가해요. 마치 자리를 정리해 주는 것과 같아요.

### 4. 많은 정보를 배워요

컴퓨터는 많은 글을 읽고 그 안에서 규칙을 찾아요. 그러면 질문을 받았을 때, 그 규칙을 바탕으로 답을 만들어낼 수 있어요. 책을 많이 읽고, 영화도 보면서 "아, 이런 식으로 대답하는구나!"를 배우는 것과 비슷하죠.

### 정리하자면

Transformer 모델은 한 단어가 다른 단어와 어떻게 연결되어 있는지를 알아보고, 단어의 순서를 기억하면서 컴퓨터가 글을 이해하고 대답을 만드는 방법이랍니다. 그래서 우리가 로봇과 대화할 때, 똑똑하게 대화할 수 있게 도와주는 것이죠!

혹시 궁금한 점이 더 있나

### 반복처리

In [17]:
# 대화내역을 로깅
messages = [
    {'role': 'system', 'content': '너는 친절한 챗봇이다.'}
]

print('종료하려면, exit를 입력하세요...')
while True:
    # 사용자 입력
    user_input = input('User: ')

    if user_input.strip().lower() == 'exit':
        print('채팅을 종료합니다...')
        break

    # messages에 사용자 입력 추가
    messages.append({'role': 'user', 'content': user_input})

    # LLM 요청
    response = client.chat.completions.create(
        model='gpt-4.1',
        messages=messages,
        temperature=1
    )
    assistant_message = response.choices[0].message.content
    print(f'Assistant: {assistant_message}')

    # messages 챗봇 출력 추가
    messages.append({'role': 'assistant','content': assistant_message})

종료하려면, exit를 입력하세요...
User: 안녕 나는 차은우야
Assistant: 안녕하세요, 차은우 님! 만나서 정말 반가워요 😊 오늘 기분은 어떠세요? 어떤 이야기든 편하게 해주세요!
User: 오늘은 비가 와서 기분이 좋지 않아ㅜ
Assistant: 비 오는 날은 괜히 마음이 우울해질 때가 있죠. 은우님, 요즘 많이 바쁘고 힘드셨을 것 같아요. 그래도 이렇게 이야기해주셔서 고마워요! 따뜻한 차 한잔 마시면서 조금 쉬어보는 건 어떨까요? 혹시 듣고 싶은 노래나 위로가 되는 말이 있다면 말해줘요. 제가 힘이 되어드릴게요. 😊
User: 이럴때 마시기 좋은 차가 있을까?
Assistant: 그럼요! 비 오는 날, 기분이 살짝 처질 때 마시면 좋은 차 몇 가지 추천해 드릴게요.

1. **캐모마일차**  
은은하고 부드러운 향이 마음을 편안하게 해줘요. 스트레스를 풀고 싶을 때 딱이죠.

2. **홍차(얼그레이, 잉글리시 브렉퍼스트 등)**  
진한 향과 맛이 비 오는 날의 쓸쓸함을 달래주고, 따듯한 느낌을 줘요. 우유를 곁들여 밀크티로 마셔도 너무 좋습니다.

3. **유자차**  
새콤달콤한 향이 기분을 상쾌하게 만들어줘요. 비타민C도 풍부해서 피로도 풀린답니다.

4. **생강차**  
몸이 따뜻해지는 느낌이 들면서 감기 예방에도 좋아요. 꿀을 조금 넣으면 더 맛있고 포근해져요.

5. **녹차**  
상쾌하고 깔끔한 맛이 마음을 진정시키는 데 도움을 줄 수 있어요.

혹시 위에 있는 차 중에 집에 있는 게 있다면, 오늘은 그 차 한 잔과 함께 여유로운 시간 가져보세요. 아니면 궁금한 차가 있으면 더 알려줄 수 있어요! 😊

비 오는 날, 은우님께 따스한 위로가 되길 바라요.
User: exit
채팅을 종료합니다...


## stream

In [19]:
stream = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{'role': 'user', 'content': "지금 stream 테스트를 할거야. 아주 긴 응답 메세지를 보내줘."}],
        stream=True
)

for chunk in stream:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end='')

물론입니다! 여기 긴 응답 메시지를 작성해 드릴게요.

---

안녕하세요! 이 메시지는 stream 테스트를 위한 긴 응답입니다. 여러 주제를 아우르며 내용을 길게 이어가 보겠습니다.

**1. 기술의 발전**
최근 몇 년간 기술은 너무나도 빠르게 발전하고 있습니다. 인공지능(AI), 머신러닝, 블록체인, 사물인터넷(IoT) 등 다양한 분야에서 혁신이 이루어지고 있으며, 이는 우리의 일상생활에도 깊은 영향을 미치고 있습니다. 예를 들어, AI는 의료, 금융, 교육 등 여러 분야에서 데이터를 분석하고 예측하는 데 사용되고 있습니다. 이러한 기술들은 우리가 이전에는 상상하지 못했던 방식으로 문제를 해결하고 있습니다.

**2. 환경 문제**
기술 발전과 함께 환경 문제도 점점 더 심각해지고 있습니다. 기후 변화, 대기 오염, 플라스틱 쓰레기 문제 등은 모두 우리가 해결해야 할 중요한 과제입니다. 이러한 문제를 해결하기 위해 많은 국가가 지속 가능한 발전 목표를 세우고 다양한 노력을 기울이고 있습니다. 재생 가능 에너지, 전기 자동차, 그리고 친환경 제품은 이러한 노력의 일환으로 주목받고 있습니다.

**3. 사회적 변화**
사회는 급속히 변화하고 있으며, 이러한 변화는 또한 기술의 발전과 관련이 있습니다. 디지털 혁명은 우리의 소통 방식, 소비 패턴, 그리고 일자리 구조를 변화시키고 있습니다. 원격 근무의 증가, 디지털 경제의 부상 등은 우리가 사회를 바라보는 시각을 새롭게 하고 있습니다. 특히, 팬데믹 이후로 많은 기업들이 원격 근무 체제로 전환하면서, 직원들의 업무 만족도와 생산성이 어떻게 변화하는지에 대한 연구도 활발히 진행되고 있습니다.

**4. 교육의 미래**
교육 분야에서도 많은 변화가 일어나고 있습니다. 전통적인 교육 방식에서 벗어나 온라인 교육, 혼합 학습, 그리고 개인 맞춤형 교육이 점점 더 중요해지고 있습니다. 이러한 변화는 학생들이 보다 효과적으로 학습할 수 있는 환경을 만들어주고 있습니다. 특히, COVID-19 팬데믹은 원격 교육의 필요

In [27]:
# stream 버젼
messages = [
    {'role': 'system', 'content': '너는 친절한 챗봇이다.'}
]

print('종료하려면, exit를 입력하세요...')
while True:
    # 사용자 입력
    user_input = input('User: ')

    if user_input.strip().lower() == 'exit':
        print('채팅을 종료합니다...')
        break

    # messages에 사용자 입력 추가
    messages.append({'role': 'user', 'content': user_input})

    # LLM 요청
    stream = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        temperature=1,
        stream=True
    )
    print(f'Assistant: ', end='', flush=True)
    for chunk in stream:
        content = chunk.choices[0].delta.content
        if content is not None:
            print(content, end='', flush=True) # 내부 buffer 사용하지 않음
    print()

    # messages 챗봇 출력 추가
    messages.append({'role': 'assistant','content': assistant_message})

종료하려면, exit를 입력하세요...
User: 안녕
Assistant: 안녕하세요! 어떻게 도와드릴까요?
User: 오늘은 몇 일이야?
Assistant: 죄송하지만, 현재 날짜를 확인할 수는 없어요. 하지만 오늘의 날짜를 확인하는 방법을 알려드릴게요! 휴대폰, 컴퓨터, 또는 달력에서 쉽게 확인하실 수 있습니다. 궁금한 다른 점이 있다면 언제든지 물어보세요!
User: exit()
Assistant: 알겠습니다! 필요할 때 언제든지 돌아와 주세요. 좋은 하루 보내세요! 😊
User: exit()
Assistant: 안녕히 계세요! 필요하신 것이 있으면 언제든지 돌아와 주세요. 좋은 하루 되세요! 😊
User: exit
채팅을 종료합니다...


# Token Counting
비용 = (입력 토큰 수 * 단가) + (출력 토큰 수 * 단가) * 월 서비스 호출 수

In [28]:
!pip install tiktoken

In [36]:
# 각 모델에 따른 토커나이져(인코딩) 가져오기
import tiktoken

gpt35 = tiktoken.encoding_for_model('gpt-3.5')
gpt4o = tiktoken.encoding_for_model('gpt-4o')
gpt41 = tiktoken.get_encoding('cl100k_base')

print(gpt35, gpt4o, gpt41)

<Encoding 'cl100k_base'> <Encoding 'o200k_base'> <Encoding 'cl100k_base'>


In [31]:
# 토큰 수 세기
print(gpt35.encode('아버지가 방에 들어가신다'))
print(gpt4o.encode('아버지가 방에 들어가신다'))

[54059, 80104, 22035, 20565, 75908, 19954, 56938, 97, 32179, 20565, 83628, 13447]
[7653, 24963, 158317, 24030, 3107, 57217, 4081, 11753, 2276]


## 토큰비용 계산하기

In [37]:
text = """
더불어민주당이 6일 한동훈 국민의힘 대표가 윤석열 대통령 탄핵에 사실상 찬성 입장을 시사하자 7일로 예정됐던 윤석열 대통령의 탄핵소추안 표결을 앞당기는 방안을 고심하고 있다. 이재명 민주당 대표는 “한 대표의 입장을 정확하게 파악하는 것이 우선”이라며 신중한 태도를 보였다.

이 대표는 이날 기자들을 만나 윤 대통령 탄핵 표결을 앞당기는 방안에 대해 “지금 저렇게 불확실한 얘기를 믿고 미리 당겨서 협의를 할 필요가 있는가, 그런 생각이 일단 든다”고 말했다. 한 대표와의 회동 가능성에 대해서는 “요청은 했는데 아직 결정을 통보받지 못했다. (한 대표 측에서) 오후에 다시 연락하자는 연락이 왔다”고 전했다.

이 대표는 또 “사실 오늘 밤이 저는 매우 위험하다고 생각이 드는데, 제가 가진 감으로 본다면 오늘 밤 새벽에 또 뭔가 일을 벌이지 않을까 그런 걱정이 들긴 한다”며 ‘2차 계엄’ 가능성을 우려했다.

민주당은 이날 오전 한동훈 대표가 “윤 대통령의 조속한 직무 집행 정지가 필요하다”며 입장을 선회하자 긴급 의원총회를 열고 당내 의견 수렴에 나섰다. 의원총회에서는 탄핵소추안 표결 시점을 앞당기는 방안도 논의될 전망이다.

노종면 원내대변인은 비공개 최고위원 간담회가 끝난 뒤 “한 대표의 입장이 보도된 이후 긴장감이 높아지고 있고, 12월 3일 당일에 짐작했던 것 이상으로 치밀하게 의원, 정치인 체포 시도가 있었던 것과 이번 내란 사태에서 매우 중요한 작전이었던 걸로 파악되고 있다”며 “윤 대통령 옹위 세력이 어떻게 나올지 모르는 상황이라고 판단해 이런 비상한 상황 인식 떄문에 긴급 의원총회를 소집했다”고 전했다.

탄핵소추안 표결 시점 변경에 대해서는 “의장실에 본회의 일정 변경을 요청한 바는 아직 없다”며 “일단 신중하고 침착하게 대응할 것이고, 지금 한 대표 쪽의 입장이 뭔지 정확하게 파악하는 것이 우선이다. 필요하면 본회의를 앞당기는 방안도 의장실과 협조해서 추진할 수 있지만 아직은 결정된 바 없다”고 밝혔다.

민주당에서는 7일 오후 7시로 예정됐던 표결을 2시간 당겨 오후 5시에 추진하는 방안도 거론된다. 박성준 원내운영수석부대표는 이날 MBC 라디오 ‘김종배의 시선집중’ 인터뷰에서 “당초 오후 7시 정도 표결을 예상했는데 5시 정도는 해야 한다고 보고 있다”며 “국민의힘에서 탄핵소추안 투표 관련 상당한 지연 전략을 펼쳐서 시간을 늦출 수 있는 상황까지 고려하고 있다”고 설명했다.
"""

encoded_text_gpt4o = gpt4o.encode(text)
encoded_text_gpt41 = gpt41.encode(text)

print("gpt-4o 토큰수: ", len(encoded_text_gpt4o))
print("gpt-4.1 토큰수: ", len(encoded_text_gpt41))



gpt-4o 토큰수:  703
gpt-4.1 토큰수:  1215


In [38]:
response_gpt4o = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'system', 'content': '너는 똑부러지는 시사/경제 전문가로써, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야.'},
        {'role': 'user', 'content': text}
    ],
    temperature=.2
)

response_gpt41 = client.chat.completions.create(
    model='gpt-4.1',
    messages=[
        {'role': 'system', 'content': '너는 똑부러지는 시사/경제 전문가로써, 제공된 뉴스기사의 핵심을 잘 요약해서 정리해주는 챗봇이야.'},
        {'role': 'user', 'content': text}
    ],
    temperature=.2
)

output_gpt4o = response_gpt4o.choices[0].message.content
output_gpt41 = response_gpt41.choices[0].message.content

print("gpt-4o 응답: ", output_gpt4o)
print("gpt-4.1 응답: ", output_gpt41)

print('\n', 'gpt-4o 토큰수: ', len(gpt4o.encode(output_gpt4o)))
print('gpt-4.1 토큰수: ', len(gpt41.encode(output_gpt41)))

gpt-4o 응답:  더불어민주당은 한동훈 국민의힘 대표가 윤석열 대통령 탄핵에 사실상 찬성하는 입장을 시사하자, 탄핵소추안 표결 시점을 앞당기는 방안을 검토하고 있습니다. 이재명 민주당 대표는 한 대표의 입장을 정확히 파악하는 것이 우선이라며 신중한 태도를 보였습니다. 민주당은 한 대표의 발언 이후 긴급 의원총회를 열어 당내 의견을 수렴하고 있으며, 표결 시점을 앞당기는 방안도 논의 중입니다. 현재로서는 본회의 일정 변경 요청은 없지만, 필요시 의장실과 협조해 추진할 수 있다고 밝혔습니다. 표결은 원래 7일 오후 7시에 예정되어 있었으나, 오후 5시로 앞당기는 방안이 거론되고 있습니다.
gpt-4.1 응답:  ### 기사 핵심 요약

**1. 민주당, 윤석열 대통령 탄핵소추안 표결 시점 앞당길지 고심**
- 한동훈 국민의힘 대표가 윤 대통령 탄핵에 사실상 찬성하는 듯한 발언을 하자, 민주당이 7일로 예정된 탄핵소추안 표결을 더 앞당기는 방안을 논의 중.
- 이재명 민주당 대표는 한동훈 대표의 진짜 입장 파악이 우선이라며 신중한 태도.

**2. 긴장 고조, 민주당 긴급 의원총회 소집**
- 한동훈 대표가 “윤 대통령의 조속한 직무 정지 필요”라고 입장 선회하자, 민주당은 긴급 의원총회를 열어 당내 의견 수렴.
- 민주당은 12월 3일 있었던 정치인 체포 시도 등 최근 상황을 매우 비상하게 인식하고 있음.

**3. 표결 시점 변경 가능성**
- 공식적으로 본회의 일정 변경을 요청하진 않았으나, 필요시 의장실과 협의해 표결을 앞당길 수 있다고 밝힘.
- 실제로 7일 오후 7시로 예정된 표결을 오후 5시로 앞당기는 방안이 거론됨. 국민의힘의 지연 전략에 대응하기 위한 조치.

**4. 이재명 대표, ‘2차 계엄’ 가능성 우려**
- 이재명 대표는 “오늘 밤에 또 뭔가 일이 벌어질 수 있다”며, 정부·여당 측의 추가 강경 대응 가능성을 경계.

---

**핵심 정리:**  
한동훈 대표의 발언으로 윤 대통령 탄핵 정국이 급변하면서, 민주당은 탄핵소추안 표결을 앞

In [42]:
# 모델별 가격(2025년 6월 기준, 1M=1,000,000 토큰)
PRICING = {
    "gpt-4.1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "gpt-4.1-mini": {
        "input": 0.40,    # $0.40 / 1M input tokens
        "output": 1.60    # $1.60 / 1M output tokens
    },
    "gpt-4.1-nano": {
        "input": 0.10,    # $0.10 / 1M input tokens
        "output": 0.40    # $0.40 / 1M output tokens
    },
    "o1": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o3": {
        "input": 2.00,    # $2.00 / 1M input tokens
        "output": 8.00    # $8.00 / 1M output tokens
    },
    "o4-mini": {
        "input": 1.10,    # $1.10 / 1M input tokens
        "output": 4.40    # $4.40 / 1M output tokens
    },
    "gpt-4o": {
        "input": 2.50,    # $2.50 / 1M input tokens
        "output": 10.00   # $10.00 / 1M output tokens
    },
    "gpt-4o-mini": {
        "input": 0.15,    # $0.15 / 1M input tokens
        "output": 0.60    # $0.60 / 1M output tokens
    }
}

def count_tokens(text, model):
    if model == 'gpt-4.1':
        encoding = tiktoken.get_encoding('cl100k_base')
    else:
        encoding = tiktoken.encoding_for_model(model)
    encoded = encoding.encode(text)
    return len(encoded)

def calc_cost(input_text, output_text, model, num_service_call=1_000_000):
    input_tokens = count_tokens(input_text, model)
    output_tokens = count_tokens(output_text, model)

    # 모델별 단가 가져오기
    price = PRICING[model]

    # 비용계산
    input_cost = (input_tokens / 1_000_000) * price['input']
    output_cost = (output_tokens / 1_000_000) * price['output']
    total_coast = (input_cost + output_cost) * num_service_call

    return total_coast


print(calc_cost(text, output_gpt4o, model='gpt-4o-mini'))
print(calc_cost(text, output_gpt41, model='gpt-4.1'))


218.84999999999997
8462.0
